In [89]:
import json
import logging
import pprint
import pandas as pd
import numpy as np

from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, \
    remove_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn.externals import joblib


logging.basicConfig(format='\r%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

ENCODING = 'iso-8859-1'
pp = pprint.PrettyPrinter(indent=4)

## Data preparation
### Utilities

In [2]:
def print_state(current, target, loading=True):
    if loading:
        state = round((current / target) * 100.0, 2)
        print('\rLoading: ' + str(state) + '%', end='')
    else:
        print(f'\rRow {current}/{target}', end='')

### Text tokenizing

In [53]:
ALLOWED_SHORT_WORDS = ['c', 'c#', 'r', '3d', '2d', '1d', '7z', 'qt']


def tokenize_corpus(corpus):
    output = []
    doc_length = len(corpus)
    for idx, text in enumerate(corpus):
        if show_state and idx % show_idx == 0:
            print_state(idx + 1, doc_length)
        output.append(tokenize_text(text))
    return output


def tokenize_text(text):
    p = preprocess_string(text, filters=[strip_tags, strip_punctuation, strip_multiple_whitespaces, remove_stopwords])
    return strip_short(p)    


def strip_short(word_list):    
    return [str(word).lower() for word in word_list if len(word) > 2 or word in ALLOWED_SHORT_WORDS]


def process_tags(file_name):
    PATH_TAGS = '../data/stacksample/Tags.csv'
    tags_df = pd.read_csv(PATH_TAGS, encoding=ENCODING)
    tags = tags_df.groupby('Id')['Tag'].apply(list)
    tags_processed = [preprocess_string((" ".join(str(x) for x in row))) for row in tags.values]
    with open(f'../data/stacksample/tokenized/{file_name}.json', 'w', encoding=ENCODING) as f:
        json.dump(tags_processed, f)
    return tags_processed


def corpus_list_to_string(tokenized_corpus):
    return [' '.join(text) for text in tokenized_corpus]


def corpus_string_to_list(string_corpus):
    return [str(text).split() for text in string_corpus]

### Data loading

In [51]:
def load_json_tags(file_name):
    with open(f'../data/stacksample/tokenized/{file_name}') as f:
        return json.load(f)
        
    
def load_json_data(file_name):
    with open(f'../data/stacksample/tokenized/{file_name}') as f:
        return json.load(f)    
    

def create_csv(data_file_name, tag_file_name, file_name):
    x = corpus_list_to_string(load_json_data(data_file_name))
    y = corpus_list_to_string(load_json_tags(tag_file_name))
    df = pd.DataFrame(data={'X': x, 'Y': y})
    df.to_csv(f'../data/stacksample/{file_name}', encoding='utf-8')


def divide_dataset(x, y, data_size=None, train_dist=0.7):
    data_size = data_size if data_size is not None else len(x)
    train_size = int(data_size * train_dist)
    return x[0:train_size], y[0:train_size], x[train_size:data_size], y[train_size:data_size]
    
    
def dataset_from_json(data_file_name, tag_file_name, data_size=None, train_dist=0.7):
    x = load_json_data(data_file_name)
    y = load_json_tags(tag_file_name)
    return divide_tataset(x, y, data_size=None, train_dist=0.7)
    
    
def dataset_from_csv(data_file_name, data_size=None, train_dist=0.7):
    df = pd.read_csv(f'../data/stacksample/{data_file_name}', nrows=data_size)
    x = df['X'].values
    y = df['Y'].values
    return divide_dataset(x, y, data_size=None, train_dist=0.7)

---

In [20]:
TAGS_FILE = 'tags.json'
DATA_FILE = 'non_stemmed.json'
CSV_FILE = 'data_tags.csv'
#create_csv(DATA_FILE, TAGS_FILE, 'data_tags.csv')
#x_train, y_train, x_test, y_test = dataset_from_json(DATA_FILE, TAGS_FILE, 200000)
x_train, y_train, x_test, y_test = dataset_from_csv(CSV_FILE, 200000)

In [54]:
y_train = corpus_string_to_list(y_train)
y_test = corpus_string_to_list(y_test)

In [57]:
print(x_train.shape, len(y_train), x_test.shape, len(y_test))

(140000,) 140000 (60000,) 60000


In [59]:
all_labels = np.hstack([y_train, y_test])
print(all_labels.shape)

(200000,)


In [60]:
mlb = MultiLabelBinarizer().fit(all_labels)

In [73]:
y_train_binary = mlb.transform(y_train)
y_test_binary = mlb.transform(y_test)
print(y_train_binary.shape , y_test_binary.shape)

(140000, 10858) (60000, 10858)


sqlstatement execute multiple queries statementi written database generation script sql want execute adobe air application create table trole roleid integer primary key rolename varchar create table tfile fileid integer primary key filename varchar filedescription varchar 500 thumbnailid integer fileformatid integer categoryid integer isfavorite boolean dateadded date globalaccesscount integer lastaccesstime date downloadcomplete boolean isnew boolean isspotlight boolean duration varchar create table tcategory categoryid integer primary key categoryname varchar parent categoryid integer execute adobe air following methods public static function runsqlfromfile filename string void var file file file applicationdirectory resolvepath filename var stream filestream new filestream stream open file filemode read var strsql string stream readutfbytes stream bytesavailable nonquery strsql public static function nonquery strsql string void var sqlconnection sqlconnection new sqlconnection sqlco

## Model building    

In [63]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [69]:
def create_classifier():    
    return OneVsRestClassifier(SGDClassifier(
        random_state=0, loss='hinge', alpha=0.00001, penalty='elasticnet'))


def create_tfidf():
    return TfidfVectorizer(min_df=0.00009, max_features=200000, stop_words='english', 
                           smooth_idf=True, norm='l2', sublinear_tf=False, use_idf=True, ngram_range=(1,3))

In [82]:
tfidf = create_vectorizer()
ovr_sgd = create_classifier()

pipeline = Pipeline([
    ('tfidf', create_tfidf()), 
    ('sgd', create_classifier())
])

d:\users\janvi\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### Training

In [83]:
pipeline.fit(x_train, y_train_binary)

d:\users\janvi\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 24 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 35 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 45 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: Us

d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 436 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 445 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 446 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 450 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 467 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 475 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 

d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1007 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1013 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1026 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1041 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1058 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1075 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1527 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1535 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1537 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1538 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1539 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 1547 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 2078 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 2089 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 2097 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 2100 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 2115 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 2118 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

...

d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9270 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9283 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9300 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9305 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9320 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9344 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9751 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9752 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9755 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9757 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9762 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 9771 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: 

d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10222 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10227 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10240 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10242 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10245 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10251 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWar

d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10602 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10615 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10616 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10620 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10624 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 10625 is present in all training examples.
  str(classes[c]))
d:\users\janvi\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWar

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=200000, min_df=9e-05,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=T..., random_state=0,
       shuffle=True, tol=None, verbose=0, warm_start=False),
          n_jobs=1))])

In [91]:
PIPELINE_NAME = 'sklearn_sgd_pipeline'
joblib.dump(pipeline, f'../data/stacksample/models/{PIPELINE_NAME}.pkl')

['../data/stacksample/models/sklearn_sgd_pipeline.pkl']

## Model evaluation

In [85]:
y_preds_binary = pipeline.predict(x_test)

d:\users\janvi\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [87]:
mean_f1 = f1_score(y_test_binary, y_preds_binary, average='micro')

In [88]:
print(mean_f1)

0.36059772761186076


### Saving the pipeline